## About

> Function as a service

Functions as a Service (FaaS) is a cloud computing model that allows developers to build, run, and manage applications without building and maintaining the underlying infrastructure. In a FaaS cloud provider, the infrastructure is managed, and developers focus only on writing the application logic in the form of functions. Service providers then perform these functions in response to events or requests.

In [1]:
import json
import numpy as np
from sklearn.datasets import load_iris


In [2]:
# aws lambda function to process data
def lambda_handler(event, context):
    iris = load_iris()
    summary_stats = {
        'mean': np.mean(iris.data, axis=0).tolist(),
        'std_dev': np.std(iris.data, axis=0).tolist(),
        'min': np.min(iris.data, axis=0).tolist(),
        'max': np.max(iris.data, axis=0).tolist(),
    }
    
    return {
        'statusCode': 200,
        'body': json.dumps(summary_stats)
    }

In [ ]:
#using boto3 to upload the function code to aws lambda and create a new function
import boto3

lambda_client = boto3.client('lambda', region_name='us-east-1')
with open('lambda_function.zip', 'rb') as f:
    response = lambda_client.create_function(
        FunctionName='iris-summary-stats',
        Runtime='python3.8',
        Role='<lambda-execution-role-arn>',
        Handler='lambda_function.lambda_handler',
        Code=dict(ZipFile=f.read()),
        Description='Calculates summary statistics of the iris dataset',
        Timeout=30,
        MemorySize=256
    )

print(response)

In [ ]:
#invoking the function and printing the output
response = lambda_client.invoke(FunctionName='iris-summary-stats')
response_payload = json.loads(response['Payload'].read())
print(response_payload)